In [1]:
%run abalone.ipynb

In [2]:
def pulsar_exec(epoch_count=10, mb_size=10, report=1):
    load_pulsar_dataset()
    init_model()
    train_and_test(epoch_count, mb_size, report)

In [3]:
def load_pulsar_dataset():
    with open('pulsar_stars.csv') as csvfile:
        csvreader = csv.reader(csvfile)
        next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    global data, input_cnt, output_cnt
    input_cnt, output_cnt = 8, 1
    data = np.asarray(rows, dtype='float32')

In [4]:
def forward_postproc(output, y): # 순전파 처리 수행
    # 두 행렬의 각 원소 짝에 대한 시그모이드 교차 엔트로피를 일괄적으로 구함
    entropy = sigmoid_cross_entropy_with_logits(y, output)
    # 이들의 평균 = loss
    loss = np.mean(entropy)
    return loss, [y, output, entropy]

def backprop_postproc(G_loss, aux):
    y, output, entropy = aux
    
     # 순전파는 entropy -> loss였음, 따라서 역전파는 loss -> entropy
    # 1.0을 행렬의 원소 수로 나누어 각 원소의 손실 기울기로 부여
    g_loss_entropy = 1.0 / np.prod(entropy.shape)
    
    # y와 output 쌍 간의 시그모이드 교차 엔트로피 편미분 값을 구함
    g_entropy_output = sigmoid_cross_entropy_with_logits_derv(y, output)    
    
    #역전파 결과를 산출
    G_entropy = g_loss_entropy * G_loss
    G_output = g_entropy_output * G_entropy
    
    return G_output

In [5]:
def eval_accuracy(output, y):
    # output의 부호에 따라서 신경망의 판단을 알 수 있음(시그모이드를 안써도됨)
    estimate = np.greater(output, 0)
    # 정답은 1 or 0 , 안전하게 0.5와 비교하여 구함
    answer = np.greater(y, 0.5)
    # 불리언값 배열로 비교
    correct = np.equal(estimate, answer)
    
    #평균을 내면 결국 불리언이 1,0으로 간주되어 계산됨
    return np.mean(correct)

In [6]:
def relu(x): # x배열 원소 각각에 대해 음수는 모두 0으로 대치하는 함수
    # np.max는 문자열 전체와 0을 비교하는 것이어서 다름
    return np.maximum(x, 0)

def sigmoid(x):
    return np.exp(-relu(-x)) / (1.0 + np.exp(-np.abs(x)))
        
def sigmoid_derv(x, y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return relu(x) - x * z + np.log(1 + np.exp(-np.abs(x)))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)